In [1]:
import pandas as pd
import numpy as np
from numpy import math
from datetime import datetime, date, time
from datetime import timedelta

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

In [2]:
header_list = ['Date Time', 'Voltage', 'Current']
df = pd.read_csv('sensors_data.csv', names = header_list)
df['Power (W)'] = df['Voltage'] * df['Current']
df['Power (KW)'] = df['Power (W)'] / 1000
df['Date Time'] = pd.to_datetime(df['Date Time'])
df['Date'] = df['Date Time'].dt.date
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Date Time']).dt.time
df['Hour'] = pd.to_datetime(df['Date Time']).dt.hour
df['Time'] = df['Time'].astype(str)
# df['Hour'] = df['Hour'].astype(str)

In [3]:
df.head(5)

,Date Time,Voltage,Current,Power (W),Power (KW),Date,Time,Hour
0,2022-06-24 23:31:26,0.26855,0.0,0.0,0.0,2022-06-24,23:31:26,23
1,2022-06-25 00:31:26,0.26855,0.0,0.0,0.0,2022-06-25,00:31:26,0
2,2022-06-25 01:31:26,0.26855,0.0,0.0,0.0,2022-06-25,01:31:26,1
3,2022-06-25 01:32:26,0.26855,0.0,0.0,0.0,2022-06-25,01:32:26,1
4,2022-06-25 01:33:26,0.31738,0.0,0.0,0.0,2022-06-25,01:33:26,1


In [4]:
rearrange_columns = ['Date Time', 'Date', 'Time', 'Hour', 'Voltage', 'Current', 'Power (W)', 'Power (KW)']
df = df[rearrange_columns]
df.tail(5)

,Date Time,Date,Time,Hour,Voltage,Current,Power (W),Power (KW)
8916,2022-07-01 18:47:50,2022-07-01,18:47:50,18,13.54980,0.00000,0.000000,0.000000
8917,2022-07-01 18:48:50,2022-07-01,18:48:50,18,13.62305,0.00914,0.124515,0.000125
8918,2022-07-01 18:49:50,2022-07-01,18:49:50,18,13.62305,0.01132,0.154213,0.000154
8919,2022-07-01 18:50:51,2022-07-01,18:50:51,18,13.62305,0.00000,0.000000,0.000000
8920,2022-07-01 18:51:51,2022-07-01,18:51:51,18,13.64746,0.00596,0.081339,0.000081


In [5]:
unique_date = df['Date'].unique()
unique_date

array(['2022-06-24T00:00:00.000000000', '2022-06-25T00:00:00.000000000',
       '2022-06-26T00:00:00.000000000', '2022-06-27T00:00:00.000000000',
       '2022-06-28T00:00:00.000000000', '2022-06-29T00:00:00.000000000',
       '2022-06-30T00:00:00.000000000', '2022-07-01T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [6]:
filter_today_values = df[df['Date'] == unique_date[-1]][['Date', 'Hour', 'Power (KW)']]
today_hourly_values = filter_today_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
length_today_hourly_values = len(today_hourly_values)
length_today_hourly_values
# today_hourly_values

19

In [7]:
filter_daily_values = df[(df['Date'] > '2022-06-24') & (df['Date'] <= unique_date[-2])][['Date', 'Hour', 'Power (KW)']]
filter_daily_values

,Date,Hour,Power (KW)
1,2022-06-25,0,0.0
2,2022-06-25,1,0.0
3,2022-06-25,1,0.0
4,2022-06-25,1,0.0
5,2022-06-25,1,0.0
...,...,...,...
7934,2022-06-30,23,0.0
7935,2022-06-30,23,0.0
7936,2022-06-30,23,0.0
7937,2022-06-30,23,0.0


In [8]:
daily_hourly_values = filter_daily_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
daily_hourly_values

,Date,Hour,Power (KW)
0,2022-06-25,0,0.0
1,2022-06-25,1,0.0
2,2022-06-25,2,0.0
3,2022-06-25,3,0.0
4,2022-06-25,4,0.0
...,...,...,...
139,2022-06-30,19,0.0
140,2022-06-30,20,0.0
141,2022-06-30,21,0.0
142,2022-06-30,22,0.0


In [9]:
# daily_values = daily_values[daily_values['Date'] > '2022-06-11']

In [10]:
# data_selection = {'Mostly sunny': 3,
#                   'Partly sunny': 2,
#                   'Partly cloudy': 2,
#                   'Intermittent clouds': 2,
#                   'Partly sunny w/ showers': 1.5,
#                   'Partly cloudy w/ showers': 1.5,
#                   'Mostly clear': 1,
#                   'Clear': 1,
#                   'Mostly cloudy w/ t-storms': 0.5,
#                   'Mostly cloudy': 0.5,
#                   'Showers': 0.5,
#                   'Cloudy': 0.5,
#                   'Thunderstorms': 0.5,
#                   'Mostly cloudy w/ showers': 0.5,
#                   'Rain': 0}

In [11]:
header_list = ['Date', 'Time', 'SolarIrradiance (W/m2)', 'weather status', 'Temp (°C)', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)',
              'Direction', 'Hum (%)', 'Visibility (km)', 'UVIndex', 'UVIndexText', 'PreProbability (%)', 'RainProbability (%)',
              'CloudCover (%)']
weather_data = pd.read_csv('hourly_weather_forecasted_data.csv', names = header_list, encoding= 'unicode_escape')
# weather_data['modified_weather_status'] = weather_data['weather status'].map(data_selection)
weather_data['code'] = pd.factorize(weather_data['weather status'])[0]
weather_data.loc[weather_data['SolarIrradiance (W/m2)'] == 0, ['Temp (°C)', 'Hum (%)', 'CloudCover (%)']] = 0
weather_data.drop(['Date', 'Time', 'SolarIrradiance (W/m2)', 'weather status', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)', 'Direction', 'Visibility (km)', 'UVIndex', 
                   'UVIndexText', 'PreProbability (%)', 'RainProbability (%)', 'Hum (%)'], axis = 1, inplace = True)
# weather_data['weather status'].unique()
weather_data.tail(14)

,Temp (°C),CloudCover (%),code
154,15.9,70,14
155,16.7,70,0
156,17.8,97,11
157,18.3,100,11
158,18.6,84,7
159,18.9,68,14
160,19.0,52,0
161,18.8,42,2
162,18.3,32,2
163,17.7,22,4


In [12]:
df1 = pd.concat([daily_hourly_values, weather_data], axis = 1)
df1.drop(['Date', 'Hour'], axis = 1, inplace = True)
# df1.loc[df1['SolarIrradiance (W/m2)'] == 0, ['Temp (°C)', 'Hum (%)', 'CloudCover (%)']] = 0
# df1.to_csv('solar_energy_predictions.csv')
df1.tail(25)

,Power (KW),Temp (°C),CloudCover (%),code
143,0.0,0.0,0,14
144,NaN,0.0,0,0
145,NaN,0.0,0,0
146,NaN,0.0,0,0
147,NaN,0.0,0,10
148,NaN,0.0,0,1
149,NaN,9.2,46,2
150,NaN,10.0,68,14
151,NaN,11.1,70,0
152,NaN,12.8,70,0


In [13]:
count_total_rows = len(df1) - 24

In [14]:
independent_columns = df1[['Temp (°C)', 'CloudCover (%)', 'code']][0:count_total_rows]

In [15]:
dependent_column = df1['Power (KW)'][0:count_total_rows]

In [16]:
reg = linear_model.LinearRegression(fit_intercept = False)
reg.fit(independent_columns, dependent_column)

LinearRegression(fit_intercept=False)

#### Is model well fit or not?

In [17]:
r_sq = reg.score(independent_columns, dependent_column)
r_sq

0.3717231433602942

In [18]:
reg.intercept_

0.0

In [19]:
reg.coef_

array([ 0.07771922, -0.00401818, -0.01244882])

In [20]:
forcasted_data = df1[['Temp (°C)', 'CloudCover (%)', 'code']].tail(24)

In [21]:
return_array = list(reg.predict(forcasted_data))

In [22]:
# predicted_data = pd.DataFrame(return_array, columns = ['Power (KW)'])
# predicted_data

In [23]:
now = datetime.now()
date = now.strftime('%Y-%m-%d')
current_date_24 = [date, date, date, date, date, date, date, date, date, date, date, date, date, date, date, date,
                date, date, date, date, date, date, date, date]
current_date_12 = [date, date, date, date, date, date, date, date, date, date, date, date]

In [24]:
hours_24 = list(daily_hourly_values['Hour'][0:24])
hours_12 = list(daily_hourly_values['Hour'][0:12])

In [25]:
data_dict = {'Date': current_date_24, 'Hour': hours_24, 'Power (KW)': return_array}

In [26]:
data_dataframe = pd.DataFrame(data_dict)
data_dataframe

,Date,Hour,Power (KW)
0,2022-07-01,0,0.000000
1,2022-07-01,1,0.000000
2,2022-07-01,2,0.000000
3,2022-07-01,3,-0.124488
4,2022-07-01,4,-0.012449
5,2022-07-01,5,0.505283
6,2022-07-01,6,0.329672
7,2022-07-01,7,0.581410
8,2022-07-01,8,0.713533
9,2022-07-01,9,0.837884


In [27]:
data_dataframe['Power (KW)'].sum()

15.521346637601393

## Evaluating the model

#### MSE = mean_squared_error(actual value, predicted value)

In [28]:
mean_sq_error = metrics.mean_squared_error(today_hourly_values['Power (KW)'], 
                                           data_dataframe['Power (KW)'].head(length_today_hourly_values))
mean_sq_error

0.2525408097555828

In [29]:
root_mean_sq_error = np.sqrt(mean_sq_error)
root_mean_sq_error

0.5025343866399421

In [30]:
mean_ab_error = metrics.mean_absolute_error(today_hourly_values['Power (KW)'], 
                                            data_dataframe['Power (KW)'].head(length_today_hourly_values))
mean_ab_error

0.3885365256017456

#### R squared

In [31]:
r_squared = metrics.r2_score(today_hourly_values['Power (KW)'], 
                             data_dataframe['Power (KW)'].head(length_today_hourly_values))
r_squared

0.127459255493924

#### Yesterday results

In [32]:
filter_last_day_values = df[df['Date'] == unique_date[-2]][['Date', 'Hour', 'Power (KW)']]
last_day_hourly_values = filter_last_day_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()

filter_yes_values = df[(df['Date'] >= '2022-06-25') & (df['Date'] <= unique_date[-3])][['Date', 'Hour', 'Power (KW)']]
yes_hourly_values = filter_yes_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
header_list = ['Date', 'Time', 'SolarIrradiance (W/m2)', 'weather status', 'Temp (°C)', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)',
              'Direction', 'Hum (%)', 'Visibility (km)', 'UVIndex', 'UVIndexText', 'PreProbability (%)', 'RainProbability (%)',
              'CloudCover (%)']
weather_data1 = pd.read_csv('hourly_weather_forecasted_data.csv', names = header_list, encoding= 'unicode_escape')
weather_unique_date = weather_data1['Date'].unique()
filter_weather_yes_values = weather_data1[
                                         (weather_data1['Date'] >= '2022-06-25') & 
                                         (weather_data1['Date'] <= weather_unique_date[-3])][['SolarIrradiance (W/m2)',
                                                                                             'Temp (°C)', 'Hum (%)',
                                                                                             'CloudCover (%)']]
yes_df1 = pd.concat([yes_hourly_values, filter_weather_yes_values], axis = 1)
yes_df1.drop(['Date', 'Hour'], axis = 1, inplace = True)
yes_df1.loc[yes_df1['SolarIrradiance (W/m2)'] == 0, ['Temp (°C)', 'Hum (%)', 'CloudCover (%)']] = 0
yes_count_total_rows = len(yes_df1)
yes_independent_columns = yes_df1[['SolarIrradiance (W/m2)', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']][0:yes_count_total_rows]
yes_dependent_column = yes_df1['Power (KW)'][0:yes_count_total_rows]
yes_reg = linear_model.LinearRegression(fit_intercept = False)
yes_reg.fit(yes_independent_columns, yes_dependent_column)
forcasted_yes_values = weather_data1[(weather_data1['Date'] == weather_unique_date[-2])][['SolarIrradiance (W/m2)',
                                                                                        'Temp (°C)', 'Hum (%)',
                                                                                        'CloudCover (%)']]
forcasted_yes_values.loc[forcasted_yes_values['SolarIrradiance (W/m2)'] == 0, ['Temp (°C)', 'Hum (%)', 'CloudCover (%)']] = 0
return_array = yes_reg.predict(forcasted_yes_values)
predicted_data = pd.DataFrame(return_array, columns = ['Power (KW)'])
mv_pe = predicted_data['Power (KW)'].sum()
mv_mse = metrics.mean_squared_error(last_day_hourly_values['Power (KW)'], predicted_data['Power (KW)'])
mv_rmse = np.sqrt(mv_mse)
mv_mae = metrics.mean_absolute_error(last_day_hourly_values['Power (KW)'], predicted_data['Power (KW)'])
mv_rs = metrics.r2_score(last_day_hourly_values['Power (KW)'], predicted_data['Power (KW)'])
mv_pe

23.727453331924817

In [33]:
# dependent_variable = 'Power (KW)'

In [34]:
# independent_variables = df1.columns.tolist()
# independent_variables.remove(dependent_variable)

In [35]:
# independent_variables

In [36]:
# X = df1[independent_variables].values

In [37]:
# y = df1[dependent_variable].values

In [38]:
# X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [39]:
# X_train

In [40]:
# regressor = LinearRegression()
# regressor.fit(X_train, y_train)

In [41]:
# y_pred = regressor.predict(X_test)
# y_pred

In [42]:
# math.sqrt(mean_squared_error(y_test, y_pred))

In [43]:
# r2_score(y_test, y_pred)